<a href="https://colab.research.google.com/github/marcelounb/Deep_Learning_with_python_JasonBrownlee/blob/master/11_1_Sonar_Object_Classi%EF%AC%81cation_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Binary Classiﬁcation Of Sonar Returns

The dataset we will use in this tutorial is the Sonar dataset. This is a dataset that describes sonar chirp returns bouncing o↵ di↵erent surfaces. The 60 input variables are the strength of the returns at di↵erent angles. It is a binary classiﬁcation problem that requires a model to di↵erentiate rocks from metal cylinders. 

It is a well understood dataset. All of the variables are continuous and generally in the range of 0 to 1. The output variable is a string M for mine and R for rock, which will need to be converted to integers 1 and 0. The dataset contains 208 observations.

A beneﬁt of using this dataset is that it is a standard benchmark problem. This means that we have some idea of the expected skill of a good model. Using cross validation, a neural network should be able to achieve performance around 84% with an upper bound on accuracy for custom models at around 88%. You can learn more about this dataset on the UCI Machine Learning repository


In [1]:
import numpy as np 
import pandas as pd 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
# load dataset 
dataframe = pd.read_csv("/content/sonar.csv", header=None) 
dataset = dataframe.values 
# split into input (X) and output (Y) variables 
X = dataset[:,0:60].astype(float) 
Y = dataset[:,60]


In [6]:
dataset[0]

array([0.02, 0.0371, 0.0428, 0.0207, 0.0954, 0.0986, 0.1539, 0.1601,
       0.3109, 0.2111, 0.1609, 0.1582, 0.2238, 0.0645, 0.066, 0.2273,
       0.31, 0.2999, 0.5078, 0.4797, 0.5783, 0.5071, 0.4328, 0.555,
       0.6711, 0.6415, 0.7104, 0.808, 0.6791, 0.3857, 0.1307, 0.2604,
       0.5121, 0.7547, 0.8537, 0.8507, 0.6692, 0.6097, 0.4943, 0.2744,
       0.051, 0.2834, 0.2825, 0.4256, 0.2641, 0.1386, 0.1051, 0.1343,
       0.0383, 0.0324, 0.0232, 0.0027, 0.0065, 0.0159, 0.0072, 0.0167,
       0.018, 0.0084, 0.009, 0.0032, 'R'], dtype=object)

In [0]:
# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)

In [7]:
X[0], Y[0]

(array([0.02  , 0.0371, 0.0428, 0.0207, 0.0954, 0.0986, 0.1539, 0.1601,
        0.3109, 0.2111, 0.1609, 0.1582, 0.2238, 0.0645, 0.066 , 0.2273,
        0.31  , 0.2999, 0.5078, 0.4797, 0.5783, 0.5071, 0.4328, 0.555 ,
        0.6711, 0.6415, 0.7104, 0.808 , 0.6791, 0.3857, 0.1307, 0.2604,
        0.5121, 0.7547, 0.8537, 0.8507, 0.6692, 0.6097, 0.4943, 0.2744,
        0.051 , 0.2834, 0.2825, 0.4256, 0.2641, 0.1386, 0.1051, 0.1343,
        0.0383, 0.0324, 0.0232, 0.0027, 0.0065, 0.0159, 0.0072, 0.0167,
        0.018 , 0.0084, 0.009 , 0.0032]), 'R')

The output variable is string values. We must convert them into integer values 0 and 1. We can do this using the LabelEncoder class from scikit-learn. This class will model the encoding required using the entire dataset via the fit() function, then apply the encoding to create a new output variable using the transform() function.


In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [9]:
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Creating the model
We are now ready to create our neural network model using Keras. We are going to use scikit-learn to evaluate the model using stratiﬁed k-fold cross validation. This is a resampling technique that will provide an estimate of the performance of the model. To use Keras models with scikit-learn, we must use the KerasClassifier wrapper. 

This class takes a function that creates and returns our neural network model. It also takes arguments that it will pass along to the call to fit() such as the number of epochs and the batch size. Let’s start o↵ by deﬁning the function that creates our baseline model. Our model will have a single fully connected hidden layer with the same number of neurons as input variables. This is a good default starting point when creating neural networks on a new problem. 

The weights are initialized using a small Gaussian random number. The Rectiﬁer activation function is used. The output layer contains a single neuron in order to make predictions. It uses the sigmoid activation function in order to produce a probability output in the range of 0 to 1 that can easily and automatically be converted to crisp class values. Finally, we are using the logarithmic loss function (binary crossentropy) during training, the preferred loss function for binary classiﬁcation problems. The model also uses the ecient Adam optimization algorithm for gradient descent and accuracy metrics will be collected when the model is trained

In [0]:
# baseline model 
def create_baseline(): 
  # create model 
  model = Sequential() 
  model.add(Dense(60, input_dim=60, kernel_initializer= 'normal' , activation= 'relu' )) 
  model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' )) 
  # Compile model 
  model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model 

# Evaluating the model
Now it is time to evaluate this model using stratiﬁed cross validation in the scikit-learn framework. We pass the number of training epochs to the KerasClassifier, again using reasonable default values. Verbose output is also turned o↵ given that the model will be created 10 times for the 10-fold cross validation being performed.

In [19]:
# evaluate model with standardized dataset 
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5) 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed, ) 
results = cross_val_score(estimator, X, encoded_Y, cv=kfold) 
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

Epoch 1/100
187/187 [==============================] - 0s 602us/step - loss: 0.6899 - accuracy: 0.5294
Epoch 2/100
187/187 [==============================] - 0s 212us/step - loss: 0.6742 - accuracy: 0.5401
Epoch 3/100
187/187 [==============================] - 0s 218us/step - loss: 0.6620 - accuracy: 0.6257
Epoch 4/100
187/187 [==============================] - 0s 211us/step - loss: 0.6435 - accuracy: 0.5989
Epoch 5/100
187/187 [==============================] - 0s 217us/step - loss: 0.6230 - accuracy: 0.6738
Epoch 6/100
187/187 [==============================] - 0s 222us/step - loss: 0.6045 - accuracy: 0.6684
Epoch 7/100
187/187 [==============================] - 0s 224us/step - loss: 0.5787 - accuracy: 0.6898
Epoch 8/100
187/187 [==============================] - 0s 212us/step - loss: 0.5559 - accuracy: 0.7540
Epoch 9/100
187/187 [==============================] - 0s 224us/step - loss: 0.5335 - accuracy: 0.7540
Epoch 10/100
187/187 [==============================] - 0s 219us/step - l